# Power series and finite differences

A standard way to uncover the accuracy of a finite-difference formula is by expansion in power series. We'll use formal power series here, which is a bit tricky. Suppose $hD$ represents the differentiation operator times a number $h$. Then define

In [1]:
R.<hD> = QQ[[]]
ser = hD + O(hD^6)
Z = ser.exp()
Z

1 + hD + 1/2*hD^2 + 1/6*hD^3 + 1/24*hD^4 + 1/120*hD^5 + O(hD^6)

We can interpret Taylor's theorem as stating that $Z$ is an operator that "shifts" a smooth function $u(x)$ to the function $u(x+h)$. Thus, for example, a forward difference operator has the series

In [2]:
fd1 = Z - 1
fd1

hD + 1/2*hD^2 + 1/6*hD^3 + 1/24*hD^4 + 1/120*hD^5 + O(hD^6)

That is,

$$
u(x+h) - u(x) & = hu'(x) + \frac{1}{2}h^2u''(x) + \cdots \\ 
\frac{u(x+h)-u(x)}{h} = u'(x) + \frac{1}{2}h u''(x) + O(h^2).
$$

Hence we say this formula is **first-order accurate**, since its error is led by the $O(h^1)$ term. Similarly, the two-term backward difference is also first-order:

In [3]:
bd1 = 1 - Z^(-1) 
bd1

hD - 1/2*hD^2 + 1/6*hD^3 - 1/24*hD^4 + 1/120*hD^5 + O(hD^6)

The centered difference, however, is second-order accurate:

In [4]:
cd2 = (Z - Z^(-1))/2
cd2

hD + 1/6*hD^3 + 1/120*hD^5 + O(hD^6)

$$
u(x+h) - u(x) & = hu'(x) + \frac{1}{2}h^2u''(x) + \cdots \\ 
\frac{u(x+h)-u(x-h)}{2h} &= u'(x) + \frac{1}{6} h^2 u'''(x) + O(h^4).
$$

One interpretation is that the antisymmetry around $h=0$ buys us an extra order.

The same series analysis can be used to derive methods as well. You decide which nodes to include, called the **stencil** of the method, give each function value an unknown coefficient, expand everything around the evaluation point, and choose the coefficients to cancel out as many terms as possible. 

There is a better method, though, that uses the connection between differentiation and shifting to derive formulas. 
Note that formally, $Z=e^{hD}$ around $h=0$, so that $hD = \log(Z)$ around $Z=1$. Hence finite difference formulas are always found as truncations of the series:

In [5]:
var('z')
taylor(log(z),z,1,4)

-1/4*(z - 1)^4 + 1/3*(z - 1)^3 - 1/2*(z - 1)^2 + z - 1

So if we were setting out to find the two-point forward difference, for example, we'd start with

$$
\frac{a_1 u(h) + a_0 u(0)}{h} & \approx u'(0) \\ 
a_1 Zu(0) + a_0u(0) & \approx hDu(0) \\ 
a_1 Z + a_0 & \approx \log(Z). 
$$

Hence,

In [6]:
expand(taylor(log(z),z,1,1))

z - 1

If we add a node at $2h$, then we have $a_2 Z^2 + a_1 Z + a_0 \approx \log(Z)$:

In [7]:
expand(taylor(log(z),z,1,2))

-1/2*z^2 + 2*z - 3/2

This corresponds to a 2nd-order forward difference on three points:

$$
\frac{-3u(0) + 4u(h) - u(2h)}{2h} = u'(0) + O(h^2). 
$$

We can handle centered (and any generally-offset) formula on evenly spaced nodes. In the centered three-point formula, for instance, we have 

$$
\frac{a_{-1}u(-h) + a_0 u(0) + a_1u(h)}{h} & \approx u'(0)  \\ 
a_{-1}Z^{-1} + a_0 + a_1 Z & \approx \log(Z) \\ 
a_{-1} + a_0 Z + a_1 Z^2 & \approx Z \log(Z) 
$$

In [8]:
expand(taylor(z*log(z),z,1,2))

1/2*z^2 - 1/2

Similarly, we can derive a five-point centered formula via

In [9]:
expand(taylor(z^2*log(z),z,1,4))

-1/12*z^4 + 2/3*z^3 - 2/3*z + 1/12

## Higher derivatives

We can play the same games for 2nd and higher derivative formulas. Here is the most famous formula for a second derivative: 3-point, centered, and 2nd-order:

In [10]:
expand(taylor(z*log(z)^2,z,1,3))

z^2 - 2*z + 1

$$
u''(0) = \frac{u_{-1}-2u_0+u_1}{h^2} + O(h^2)
$$

This can be carried out to 4th-order using 5 points:

In [11]:
expand(taylor(z^2*log(z)^2,z,1,5))

-1/12*z^4 + 4/3*z^3 - 5/2*z^2 + 4/3*z - 1/12

We can also do asymmetric (forward and backward) formulas:

In [12]:
expand(taylor(log(z)^2,z,1,3))

-z^3 + 4*z^2 - 5*z + 2

$$
u''(0) = \frac{2u_{0}-5u_1+4u_2 - u_3}{h^2} + O(h^2)
$$